In [19]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from datetime import datetime
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt

# Conectar con MetaTrader 5
if not mt5.initialize():
    print("Error al conectarse a MetaTrader 5")
    mt5.shutdown()

# Establecer el símbolo del par de divisas y la temporalidad de 4 horas
symbol = "EURAUD"
timeframe = mt5.TIMEFRAME_H4

# Calcular la cantidad de barras necesarias para los últimos 3 meses
months = 3
days_per_month = 30
bars_needed = int(months * days_per_month * 24 / 4)

# Descargar los datos del par de divisas
data = mt5.copy_rates_from(symbol, timeframe, datetime.now(), bars_needed)

# Cerrar la conexión con MetaTrader 5
mt5.shutdown()

True

In [20]:
# Crear un DataFrame de Pandas con los datos descargados
df = pd.DataFrame(data)
df['time'] = pd.to_datetime(df['time'], unit='s')
df = df.set_index('time')

In [21]:
display(df)

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2023-01-12 20:00:00,1.55523,1.55818,1.55467,1.55727,27714,0,0
2023-01-13 00:00:00,1.55611,1.55906,1.55485,1.55780,15632,0,0
2023-01-13 04:00:00,1.55780,1.56081,1.55769,1.55864,16353,0,0
2023-01-13 08:00:00,1.55864,1.55888,1.55239,1.55433,29286,0,0
2023-01-13 12:00:00,1.55435,1.56047,1.55223,1.55862,28740,0,0
...,...,...,...,...,...,...,...
2023-05-18 00:00:00,1.62712,1.62883,1.62531,1.62881,8369,0,0
2023-05-18 04:00:00,1.62879,1.63491,1.62694,1.62874,19067,0,0
2023-05-18 08:00:00,1.62874,1.63171,1.62709,1.63074,23596,0,0


In [22]:
# Crear una lista para almacenar los cambios diarios por bloque de 6 líneas
cambios_diarios_6_lineas = []

# Iterar a través de los bloques de 6 líneas
for i in range(0, len(df), 6):
    sub_df = df.iloc[i:i+6]

    # Calcular el cambio diario para el bloque actual
    cambios_diarios = sub_df['close'].diff()

    # Agregar los cambios diarios a la lista correspondiente
    cambios_diarios_6_lineas.extend(cambios_diarios)

# Asegurar que la longitud de los cambios coincida con la longitud del índice del DataFrame
cambios_diarios_6_lineas.extend([0] * (len(df) - len(cambios_diarios_6_lineas)))

# Agregar los cambios diarios en una nueva columna
df['Cambios Diarios'] = cambios_diarios_6_lineas

# Mostrar el DataFrame con los cambios diarios
print(df)

                        open     high      low    close  tick_volume  spread   
time                                                                           
2023-01-12 20:00:00  1.55523  1.55818  1.55467  1.55727        27714       0  \
2023-01-13 00:00:00  1.55611  1.55906  1.55485  1.55780        15632       0   
2023-01-13 04:00:00  1.55780  1.56081  1.55769  1.55864        16353       0   
2023-01-13 08:00:00  1.55864  1.55888  1.55239  1.55433        29286       0   
2023-01-13 12:00:00  1.55435  1.56047  1.55223  1.55862        28740       0   
...                      ...      ...      ...      ...          ...     ...   
2023-05-18 00:00:00  1.62712  1.62883  1.62531  1.62881         8369       0   
2023-05-18 04:00:00  1.62879  1.63491  1.62694  1.62874        19067       0   
2023-05-18 08:00:00  1.62874  1.63171  1.62709  1.63074        23596       0   
2023-05-18 12:00:00  1.63074  1.63118  1.62664  1.62959        21239       0   
2023-05-18 16:00:00  1.62959  1.63164  1

In [23]:
# Calcular las ganancias y pérdidas diarias para cada 6 líneas del DataFrame
df['Ganancias Diarias'] = df['Cambios Diarios'].rolling(6).apply(lambda x: x[x > 0].sum(), raw=True)
df['Pérdidas Diarias'] = df['Cambios Diarios'].rolling(6).apply(lambda x: x[x < 0].sum(), raw=True)

# Mostrar el DataFrame con las ganancias y pérdidas diarias
print(df)

                        open     high      low    close  tick_volume  spread   
time                                                                           
2023-01-12 20:00:00  1.55523  1.55818  1.55467  1.55727        27714       0  \
2023-01-13 00:00:00  1.55611  1.55906  1.55485  1.55780        15632       0   
2023-01-13 04:00:00  1.55780  1.56081  1.55769  1.55864        16353       0   
2023-01-13 08:00:00  1.55864  1.55888  1.55239  1.55433        29286       0   
2023-01-13 12:00:00  1.55435  1.56047  1.55223  1.55862        28740       0   
...                      ...      ...      ...      ...          ...     ...   
2023-05-18 00:00:00  1.62712  1.62883  1.62531  1.62881         8369       0   
2023-05-18 04:00:00  1.62879  1.63491  1.62694  1.62874        19067       0   
2023-05-18 08:00:00  1.62874  1.63171  1.62709  1.63074        23596       0   
2023-05-18 12:00:00  1.63074  1.63118  1.62664  1.62959        21239       0   
2023-05-18 16:00:00  1.62959  1.63164  1

In [15]:
import numpy as np

# Obtener los últimos 14 días de datos
ultimos_14_dias = df[-14 * 6:]

# Crear una lista para almacenar los promedios de ganancias y pérdidas por bloque de 6 líneas
ganancias_promedio_6_lineas = []
pérdidas_promedio_6_lineas = []

# Iterar a través de los bloques de 6 líneas
for i in range(0, len(ultimos_14_dias), 6):
    sub_df = ultimos_14_dias.iloc[i:i+6]

    # Calcular las ganancias y pérdidas de cada bloque de 6 líneas
    ganancias_diarias = sub_df['Ganancias Diarias'].sum(skipna=True)
    pérdidas_diarias = sub_df['Pérdidas Diarias'].sum(skipna=True)

    # Calcular los promedios de ganancias y pérdidas para el bloque actual
    if not np.isnan(ganancias_diarias) and not np.isnan(pérdidas_diarias):
        ganancias_promedio = ganancias_diarias.mean()
        pérdidas_promedio = pérdidas_diarias.mean()
    else:
        ganancias_promedio = 0
        pérdidas_promedio = 0

    # Agregar los promedios a la lista correspondiente
    ganancias_promedio_6_lineas.append(ganancias_promedio)
    pérdidas_promedio_6_lineas.append(pérdidas_promedio)

# Asegurar que la longitud de los promedios coincida con la longitud del índice del DataFrame
ganancias_promedio_6_lineas.extend([0] * (len(df) - len(ganancias_promedio_6_lineas)))
pérdidas_promedio_6_lineas.extend([0] * (len(df) - len(pérdidas_promedio_6_lineas)))

# Agregar los promedios de ganancias y pérdidas en una nueva columna
df['Ganancias Promedio'] = ganancias_promedio_6_lineas
df['Pérdidas Promedio'] = pérdidas_promedio_6_lineas

# Mostrar el DataFrame con la nueva columna de promedios
print(df)

                        open     high      low    close  tick_volume  spread   
time                                                                           
2023-01-12 20:00:00  1.55523  1.55818  1.55467  1.55727        27714       0  \
2023-01-13 00:00:00  1.55611  1.55906  1.55485  1.55780        15632       0   
2023-01-13 04:00:00  1.55780  1.56081  1.55769  1.55864        16353       0   
2023-01-13 08:00:00  1.55864  1.55888  1.55239  1.55433        29286       0   
2023-01-13 12:00:00  1.55435  1.56047  1.55223  1.55862        28740       0   
...                      ...      ...      ...      ...          ...     ...   
2023-05-18 00:00:00  1.62712  1.62883  1.62531  1.62881         8369       0   
2023-05-18 04:00:00  1.62879  1.63491  1.62694  1.62874        19067       0   
2023-05-18 08:00:00  1.62874  1.63171  1.62709  1.63074        23596       0   
2023-05-18 12:00:00  1.63074  1.63118  1.62664  1.62959        21239       0   
2023-05-18 16:00:00  1.62959  1.63164  1

In [18]:
# Crear una lista para almacenar los promedios de precio por bloque de 6 líneas
promedio_precio_6_lineas = []

# Iterar a través de los bloques de 6 líneas
for i in range(0, len(ultimos_14_dias), 6):
    sub_df = ultimos_14_dias.iloc[i:i+6]

    # Calcular el promedio del precio para el bloque actual
    promedio_precio = sub_df['close'].mean()

    # Agregar el promedio a la lista correspondiente
    promedio_precio_6_lineas.append(promedio_precio)

# Asegurar que la longitud de los promedios coincida con la longitud del índice del DataFrame
promedio_precio_6_lineas.extend([0] * (len(df) - len(promedio_precio_6_lineas)))

# Agregar los promedios de precio en una nueva columna
df['Promedio Precio'] = promedio_precio_6_lineas

# Mostrar el DataFrame con la nueva columna de promedio de precio
display(df)

,open,high,low,close,tick_volume,spread,real_volume,Cambios Diarios,Ganancias Diarias,Pérdidas Diarias,Ganancias Promedio,Pérdidas Promedio,Promedio Precio
time,,,,,,,,,,,,,
2023-01-12 20:00:00,1.55523,1.55818,1.55467,1.55727,27714,0,0,NaN,NaN,NaN,0.0,0.0,1.658682
2023-01-13 00:00:00,1.55611,1.55906,1.55485,1.55780,15632,0,0,0.00053,NaN,NaN,0.0,0.0,1.646765
2023-01-13 04:00:00,1.55780,1.56081,1.55769,1.55864,16353,0,0,0.00084,NaN,NaN,0.0,0.0,1.654292
2023-01-13 08:00:00,1.55864,1.55888,1.55239,1.55433,29286,0,0,-0.00431,NaN,NaN,0.0,0.0,1.655522
2023-01-13 12:00:00,1.55435,1.56047,1.55223,1.55862,28740,0,0,0.00429,NaN,NaN,0.0,0.0,1.639562
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-18 00:00:00,1.62712,1.62883,1.62531,1.62881,8369,0,0,0.00153,NaN,NaN,0.0,0.0,0.000000
2023-05-18 04:00:00,1.62879,1.63491,1.62694,1.62874,19067,0,0,-0.00007,NaN,NaN,0.0,0.0,0.000000
2023-05-18 08:00:00,1.62874,1.63171,1.62709,1.63074,23596,0,0,0.00200,NaN,NaN,0.0,0.0,0.000000


In [17]:
import plotly.graph_objects as go

# Obtener los últimos 25 días de datos
ultimos_25_dias = df[-25 * 6:]

# Crear el gráfico de velas
fig = go.Figure(data=[go.Candlestick(x=ultimos_25_dias.index,
                                     open=ultimos_25_dias['open'],
                                     high=ultimos_25_dias['high'],
                                     low=ultimos_25_dias['low'],
                                     close=ultimos_25_dias['close'])])

# Agregar líneas horizontales para el promedio del precio de 14 días, precio al 70% y precio al 30%
fig.add_trace(go.Scatter(x=ultimos_25_dias.index,
                         y=[promedio_precio_14_dias] * len(ultimos_25_dias),
                         mode='lines',
                         name='Promedio Precio 14 días'))

precio_70_percent = promedio_precio_14_dias + ganancias_promedio_14_dias * 0.7
fig.add_shape(type="line",
              x0=ultimos_25_dias.index[0],
              x1=ultimos_25_dias.index[-1],
              y0=precio_70_percent,
              y1=precio_70_percent,
              line=dict(color='red', width=1),
              name='Precio al 70%')

precio_30_percent = promedio_precio_14_dias - pérdidas_promedio_14_dias * 0.3
fig.add_shape(type="line",
              x0=ultimos_25_dias.index[0],
              x1=ultimos_25_dias.index[-1],
              y0=precio_30_percent,
              y1=precio_30_percent,
              line=dict(color='green', width=1),
              name='Precio al 30%')

# Actualizar las etiquetas del eje x para que se muestren las fechas
fig.update_layout(xaxis_rangeslider_visible=False,
                  xaxis=dict(type='date'))

# Mostrar el gráfico
fig.show()

NameError: name 'promedio_precio_14_dias' is not defined

In [46]:
# Obtener los últimos 25 días de datos
ultimos_25_dias = df[-25 * 6:]

# Crear el gráfico de velas
fig = go.Figure(data=[go.Candlestick(x=ultimos_25_dias.index,
                                     open=ultimos_25_dias['open'],
                                     high=ultimos_25_dias['high'],
                                     low=ultimos_25_dias['low'],
                                     close=ultimos_25_dias['close'])])

# Agregar líneas horizontales para el promedio del precio de 14 días, precio al 70% y precio al 30%
fig.add_trace(go.Scatter(x=ultimos_25_dias.index,
                         y=[promedio_precio_14_dias] * len(ultimos_25_dias),
                         mode='lines',
                         name='Promedio Precio 14 días'))

for i in range(len(ultimos_25_dias) // 6):
    start_idx = i * 6
    end_idx = (i + 1) * 6 - 1

    precio_70_percent = promedio_precio_14_dias + ganancias_promedio_14_dias * 0.7
    fig.add_shape(type="line",
                  x0=ultimos_25_dias.index[start_idx],
                  x1=ultimos_25_dias.index[end_idx],
                  y0=precio_70_percent,
                  y1=precio_70_percent,
                  line=dict(color='red', width=1))

    precio_30_percent = promedio_precio_14_dias - pérdidas_promedio_14_dias * 0.3
    fig.add_shape(type="line",
                  x0=ultimos_25_dias.index[start_idx],
                  x1=ultimos_25_dias.index[end_idx],
                  y0=precio_30_percent,
                  y1=precio_30_percent,
                  line=dict(color='green', width=1))

# Actualizar las etiquetas del eje x para que se muestren las fechas
fig.update_layout(xaxis_rangeslider_visible=False,
                  xaxis=dict(type='date'))

# Mostrar el gráfico
fig.show()

In [56]:
# Obtener los últimos 25 días de datos
ultimos_25_dias = df[-25 * 6:]

# Filtrar las fechas de fin de semana
df_filtered = ultimos_25_dias[ultimos_25_dias.index.weekday < 5]  # Excluye sábado (5) y domingo (6)

# Eliminar las fechas de fin de semana y crear una nueva columna de índice numérico
df_filtered = df_filtered.reset_index()

# Crear el gráfico de velas
fig = go.Figure(data=[go.Candlestick(x=df_filtered.index,
                                     open=df_filtered['open'],
                                     high=df_filtered['high'],
                                     low=df_filtered['low'],
                                     close=df_filtered['close'])])

# Iterar y calcular las líneas horizontales cada 6 líneas
for i in range(0, len(df_filtered), 6):
    sub_df = df_filtered.iloc[i:i+6]

    # Calcular el promedio del precio de 14 días
    promedio_precio_14_dias = df_filtered['close'].mean()

    # Calcular el precio al 70%
    precio_70_percent = promedio_precio_14_dias + ganancias_promedio_14_dias * 0.7

    # Calcular el precio al 30%
    precio_30_percent = promedio_precio_14_dias - pérdidas_promedio_14_dias * 0.3

    # Agregar líneas horizontales
    fig.add_shape(type="line",
                  x0=df_filtered.index[0],
                  x1=df_filtered.index[-1],
                  y0=promedio_precio_14_dias,
                  y1=promedio_precio_14_dias,
                  line=dict(color='blue', width=1))

    fig.add_shape(type="line",
                  x0=df_filtered.index[0],
                  x1=df_filtered.index[-1],
                  y0=precio_70_percent,
                  y1=precio_70_percent,
                  line=dict(color='red', width=1))

    fig.add_shape(type="line",
                  x0=df_filtered.index[0],
                  x1=df_filtered.index[-1],
                  y0=precio_30_percent,
                  y1=precio_30_percent,
                  line=dict(color='green', width=1))

# Actualizar las etiquetas del eje x para que se muestren las fechas
fig.update_layout(xaxis_rangeslider_visible=False,
                  xaxis=dict(type='date'))

# Mostrar el gráfico
fig.show()